## RAG Langchain Q&A Pipeline

### Import packages

In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from rag_utils import data_loader, create_retriever
from langchain_utils import get_agent_executor

### Load the data and create chunks

In [4]:
chuncks = data_loader("../prompts/context.txt")
len(chuncks)

Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 3416, which is longer than the specified 500
Created a chunk of size 4175, which is longer than the specified 500
Created a chunk of size 4624, which is longer than the specified 500


Data loaded to vector database successfully


9

### Create the retriver

In [5]:
retriever = create_retriever(chuncks)

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Retriever created successfully.


/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [6]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [7]:
retriever.get_relevant_documents("Who are the parties to the Agreement and what are their defined names?")

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


[Document(page_content='Whereas, Advisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and\n\nWhereas, Advisor has agreed to provide the Company with such services, subject to the terms set forth in this Agreement.\n\nNOW THEREFORE THE PARTIES AGREE AS FOLLOWS:', metadata={'source': '../prompts/context.txt'}),
 Document(page_content="The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\nIn addition, the Company shall reimburse Advisor for any reasonable and actual expenses incurred by the Advisor in connection with the performance of the Services, all in accordance with the Company’s policies as determined by the Board of Directors from time to time, and subject to Company's pri

### Create langchain executor agent

In [8]:
llm_agent = get_agent_executor(retriever)

In [9]:
llm_agent.run("Who are the parties to the Agreement and what are their defined names?")

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To answer this question, I would need to know which specific agreement is being referred to. Without the context of a specific agreement, it's impossible to determine the parties involved or their defined names. Since I don't have the context, I cannot proceed with an action to find the answer.

Thought: I now know the final answer.
Final Answer: To identify the parties to an agreement and their defined names, I would need to know the specific agreement in question. Without this information, it is not possible to provide an answer. Please provide more context or specify the agreement you are referring to.

> Finished chain.


/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'To identify the parties to an agreement and their defined names, I would need to know the specific agreement in question. Without this information, it is not possible to provide an answer. Please provide more context or specify the agreement you are referring to.'

In [10]:
llm_agent.run("What is the termination notice?")



> Entering new AgentExecutor chain...


/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Thought: The termination notice could refer to several things, such as employment termination, lease termination, or service agreement termination. It generally means a written notification that an agreement or service is being ended. To provide a more accurate and detailed answer, I should use the tool to gather more information about the context of termination notice.

Action: too
Action Input: What is a termination notice?
Observation: 
Thought:

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Thought: It seems there was an issue with the action, as I did not receive any observation. I will attempt the action again to gather the necessary information.

Action: too
Action Input: What is a termination notice?
Observation: 
Thought:It appears that there is a technical issue preventing me from obtaining the information through the tool. However, based on my existing knowledge, I can provide a general answer to the question.

Final Answer: A termination notice is a formal written statement indicating the intention to end an agreement or relationship. This could be in the context of employment, where an employer informs an employee that their job will be ending; in a rental agreement, where a landlord tells a tenant that their lease will not be renewed or will be terminated; or in various service agreements, where one party informs the other that they wish to discontinue the service. The specifics of a termination notice, such as the required notice period and the form it must tak

/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'A termination notice is a formal written statement indicating the intention to end an agreement or relationship. This could be in the context of employment, where an employer informs an employee that their job will be ending; in a rental agreement, where a landlord tells a tenant that their lease will not be renewed or will be terminated; or in various service agreements, where one party informs the other that they wish to discontinue the service. The specifics of a termination notice, such as the required notice period and the form it must take, can vary depending on the context and the governing laws or contractual terms.'

In [11]:
llm_agent.run("What are the payments to the Advisor under the Agreement?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I would need to have access to the specific "Agreement" being referenced. Since I do not have access to external documents or the ability to review specific contracts, I cannot provide the details of the payments to the Advisor under the Agreement without more context or information.

Final Answer: I am unable to provide the payments to the Advisor under the Agreement without access to the specific document or additional details. Please provide more context or the relevant sections of the Agreement for me to assist you further.

> Finished chain.


/Users/ekramkedir/Desktop/real/Contract-Advisor-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'I am unable to provide the payments to the Advisor under the Agreement without access to the specific document or additional details. Please provide more context or the relevant sections of the Agreement for me to assist you further.'

: 